In [246]:
import pandas as pd
import pandas_flavor as pf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import janitor
from janitor import remove_columns, rename_column, remove_empty, filter_on, groupby_agg, clean_names

In [247]:
gdp_df = pd.read_csv("../data/UNdata_Export_20230915_015125007.csv")


cleaned_gdp_df = (
    pd.read_csv("../data/UNdata_Export_20230915_015125007.csv")
    .clean_names()
    .remove_columns("value_footnotes")
    .rename(columns={"country_or_area": "Country", "year": "Year", "value": "GDP_Per_Capita"})
    .remove_empty()
)
cleaned_gdp_df

,Country,Year,GDP_Per_Capita
0,Afghanistan,2020,1970.560169
1,Afghanistan,2019,2065.036235
2,Afghanistan,2018,2033.804389
3,Afghanistan,2017,2058.400221
4,Afghanistan,2016,2057.067978
...,...,...,...
7171,Zimbabwe,1994,2603.927369
7172,Zimbabwe,1993,2420.092791
7173,Zimbabwe,1992,2437.152908
7174,Zimbabwe,1991,2733.696626


In [248]:
birth_rate_df = (
    pd.read_excel('../data/birth_rate.xls', skiprows=3)
    .clean_names()
    .rename_column('crude_birth_rate\n_per_thousand_', 'birth_rate_per_thousand')
    .drop(0)
)

In [249]:
columns_to_keep = ['country', 'number_of_births', 'birth_rate_per_thousand', 'period']
birth_rate_df = birth_rate_df[columns_to_keep]
birth_rate_df

,country,number_of_births,birth_rate_per_thousand,period
1,Afghanistan,685401.0,48.1,..
2,Afghanistan,941680.0,41.2,Around 2005
3,Afghanistan,1088568.0,35.6,Latest
4,Albania,69507.0,32.5,Around 1970
5,Albania,77535.0,26.2,Around 1985
...,...,...,...,...
1084,Zimbabwe,236704.0,47.0,Around 1970
1085,Zimbabwe,311048.0,39.5,Around 1985
1086,Zimbabwe,357577.0,31.8,Around 1995
1087,Zimbabwe,390534.0,31.0,Around 2005


In [255]:
fertility_rate_df = (
    pd.read_excel('../data/fertility_rates.xls', skiprows=3)
    .clean_names()
    .drop(0)
)

In [256]:
columns_to_keep = ['country', 'total_fertility', 'mean_age_at_childbearing_', 'period']
fertility_rate_df = fertility_rate_df[columns_to_keep]

fertility_rate_df = fertility_rate_df.query("period != '..' and total_fertility != '..' and mean_age_at_childbearing_ != '..'")
fertility_rate_df.dropna(subset=['period'], inplace=True)
fertility_rate_df.tail(10)

,country,total_fertility,mean_age_at_childbearing_,period
1063,Yemen,5.16,30.2,Latest
1064,Zambia,6.65,31.6,Around 1970
1066,Zambia,6.08,28.8,Around 1995
1067,Zambia,5.87,28.9,Around 2005
1068,Zambia,6.16,29.3,Latest
1069,Zimbabwe,6.74,31.2,Around 1970
1070,Zimbabwe,5.42,29.5,Around 1985
1071,Zimbabwe,3.98,28.3,Around 1995
1072,Zimbabwe,3.8,28.1,Around 2005
1073,Zimbabwe,4.11,27.9,Latest


In [257]:
fertility_rate_df[fertility_rate_df['period'].isna()]

,country,total_fertility,mean_age_at_childbearing_,period


In [258]:
latest_year = '2020'
fertility_rate_df['period'] = fertility_rate_df['period'].replace('Latest', latest_year)

fertility_rate_df['period'] = fertility_rate_df['period'].str.extract('(\d+)')

fertility_rate_df = fertility_rate_df.astype({'total_fertility': 'float', 'mean_age_at_childbearing_': 'float', 'period': 'int'})
fertility_rate_df


,country,total_fertility,mean_age_at_childbearing_,period
1,Afghanistan,8.21,30.0,1970
3,Afghanistan,7.52,30.1,2005
4,Afghanistan,5.11,29.1,2020
5,Albania,5.16,29.7,1970
6,Albania,3.33,28.8,1985
...,...,...,...,...
1069,Zimbabwe,6.74,31.2,1970
1070,Zimbabwe,5.42,29.5,1985
1071,Zimbabwe,3.98,28.3,1995
1072,Zimbabwe,3.80,28.1,2005


In [264]:
print(fertility_rate_df.dtypes)

pivot_fertility_df = fertility_rate_df.pivot(index='country', columns='period', values='total_fertility')
# pivot_fertility_df['diff'] = pivot_fertility_df[2020] - pivot_fertility_df[1970]
pivot_fertility_df.head(20)


# fertility_rate_df.dropna(subset=['1970'], inplace=True)
# top_10 = pivot_fertility_df.nlargest(10, 'diff')
# top_10

country                       object
total_fertility              float64
mean_age_at_childbearing_    float64
period                         int64
dtype: object


period,1970,1985,1995,2005,2020
country,,,,,
Afghanistan,8.21,NaN,NaN,7.52,5.11
Albania,5.16,3.33,2.86,1.61,1.33
Algeria,7.86,5.35,4.38,2.38,2.74
American Samoa,5.83,4.18,3.97,4.10,3.38
Andorra,NaN,NaN,NaN,1.27,1.25
Angola,NaN,NaN,6.91,5.79,5.80
Anguilla,NaN,2.91,1.84,1.44,1.54
Antigua and Barbuda,2.68,1.86,2.20,2.36,2.01
Argentina,3.07,2.96,2.56,2.38,2.38
